In [1]:
import threading
from scapy.all import *
import psutil
import pymysql

In [2]:
class packet_sniffer():
    def __init__(self, sniffing_duration, start_id):
        self.sniffed = scapy.plist.PacketList
        self.sniffed_len = 0
        self.sniffing_duration = sniffing_duration
        self.start_id = start_id
        self.sniffed_info = dict()
        self.sniffed_info['idpackets'] = list()
        self.sniffed_info['timestamp'] = list()
        self.sniffed_info['sip'] = list()
        self.sniffed_info['dip'] = list()
        self.sniffed_info['sport'] = list()
        self.sniffed_info['dport'] = list()
        self.sniffed_info['protocol'] = list()
        self.sniffed_info['process'] = list()
        self.sniffed_info['payload'] = list()
        self.sql = str()
        self.db_rows = str()
        
    def set_data(self, host, user, password, db, charset):
        self.sniffed_len = len(self.sniffed)
        for i in range(self.sniffed_len):
            self.sniffed_info['idpackets'].append(self.start_id)
            self.start_id += 1
            self.sniffed_info['timestamp'].append(self.sniffed[i].time)
            try:
                self.sniffed_info['sip'].append(self.sniffed[i]['IP'].src)
            except IndexError:
                self.sniffed_info['sip'].append('')
            try:
                self.sniffed_info['dip'].append(self.sniffed[i]['IP'].dst)
            except IndexError:
                self.sniffed_info['dip'].append('')
            try:
                self.sniffed_info['sport'].append(str(self.sniffed[i].sport))
            except AttributeError:
                self.sniffed_info['sport'].append('')
            try:
                self.sniffed_info['dport'].append(str(self.sniffed[i].dport))
            except AttributeError:
                self.sniffed_info['dport'].append('')
            try:
                self.sniffed_info['protocol'].append(str(self.sniffed[i].proto))
            except AttributeError:
                self.sniffed_info['protocol'].append('')
            self.sniffed_info['process'].append('')
            self.sniffed_info['payload'].append('')
            
        self.conn = pymysql.connect(host=host, user=user, password=password, db=db, charset=charset)
        self.curs = self.conn.cursor()
        return (self.sniffed_len, [self.sniffed_info['idpackets'], self.sniffed_info['timestamp'], \
                                   self.sniffed_info['sip'], self.sniffed_info['dip'], self.sniffed_info['sport'], \
                                   self.sniffed_info['dport'], self.sniffed_info['protocol'], \
                                   self.sniffed_info['process'], self.sniffed_info['payload']])
        
    def start_sniffing(self):
        try:
            self.sniffed = sniff(timeout=self.sniffing_duration, filter="udp", prn=lambda x:x.summary())
        except KeyboardInterrupt:
            print('Interrupted')
            #connclose
            sys.exit(0)
            
    def append_process_name_and_payload(self):
        for dip_index in range(len(self.sniffed_info['dip'])):
            dip_value = "raddr=('" + self.sniffed_info['dip'][dip_index] + "'"
            for proc in psutil.process_iter():
                try:
                    if dip_value in str(proc.connections()):
                        self.sniffed_info['process'][dip_index] = proc.name()
                        try:
                            self.sniffed_info['payload'][dip_index] = str(self.sniffed[dip_index]['TCP'].payload)
                        except IndexError:
                            self.sniffed_info['payload'][dip_index] = str(self.sniffed[dip_index]['UDP'].payload)
                except psutil.AccessDenied:
                    pass
        print()
        print("----------------------------------------------------------------------------------------------------------")
        print("--------------------sniffed_info--------------------------------------------------------------------------")
        print('idpackets : ', self.sniffed_info['idpackets'])
        print('timestamp : ', self.sniffed_info['timestamp'])
        print('sip : ', self.sniffed_info['sip'])
        print('dip : ', self.sniffed_info['dip'])
        print('sport : ', self.sniffed_info['sport'])
        print('dport : ', self.sniffed_info['dport'])
        print('protocol : ', self.sniffed_info['protocol'])
        print('process : ', self.sniffed_info['process'])
        print('payload : ', self.sniffed_info['payload'])
        print("----------------------------------------------------------------------------------------------------------")
                        
    def store_in_db(self, sql, idx, timestamp, sip, dip, sport, dport, protocol, process, payload):
        self.sql = sql
        self.curs.execute(self.sql, (idx, timestamp, sip, dip, sport, dport, protocol, process, payload))
        
        #print(sniffed_info_)
        
    def show_db(self, sql):
        self.conn.commit()
        self.sql = sql
        self.curs.execute(self.sql)
        self.db_rows = self.curs.fetchall()
        return list(self.db_rows)
        
    def close_db(self):
        return self.conn.close()

In [6]:
if __name__ == '__main__':
    sniffer = packet_sniffer(10, 0)
    sniffer.start_sniffing()
    sniffed_len, sniffed_list = sniffer.set_data(host='localhost', user='root', password='Bong1011@@', db='testdb', \
                                                  charset='utf8')
    #sniffed_list_len = len(sniffed_list) # 9
    #print(sniffed_list_len)
    sniffer.append_process_name_and_payload()
    for i in range(sniffed_len):
        sniffer.store_in_db(sql = "insert into packets(idpackets, timestamp, sip, dip, sport, dport, \
                                protocol, process, payload) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)", \
                            idx = sniffed_list[0][i], timestamp = sniffed_list[1][i], sip = sniffed_list[2][i], \
                            dip = sniffed_list[3][i], sport = sniffed_list[4][i], dport = sniffed_list[5][i], \
                            protocol = sniffed_list[6][i], process = sniffed_list[7][i], payload = sniffed_list[8][i])
    print()
    print("----------------------------------------------------------------------------------------------------------")
    print("--------------------database_info-------------------------------------------------------------------------")
    for row in sniffer.show_db("select * from packets"):
        print(row)
    print("----------------------------------------------------------------------------------------------------------")
    sniffer.close_db()

Ether / IPv6 / UDP fe80::ea3a:12ff:fe0c:c5a7:mdns > ff02::fb:mdns / Raw
Ether / IP / UDP 172.19.87.148:52473 > 224.0.0.253:teredo / Raw
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IP / UDP 172.19.87.134:mdns > 224.0.0.251:mdns / Raw
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IP / UDP 172.19.87.27:62161 > 203.233.37.207:https / Raw
Ether / IP / UDP 172.19.83.113:netbios_ns > 172.19.83.255:netbios_ns / NBNSQueryRequest
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IP / UDP 172.19.87.27:62161 > 203.233.37.207:https / Raw
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IPv6 / UDP fe80::fcd4:36e3:dc38:85e7:55703 > ff02::1:3:llmnr / LLMNRQuery
Ether / IP / UDP 172.19.88.67:55703 > 224.0.0.252:llmnr / LLMNRQuery
Ether / IP / UDP 203.233.37.207:https > 172.19.87.27:62161 / Raw
Ether / IP / UDP 203.233.37.207:https > 172.

Ether / IP / UDP 172.19.89.16:mdns > 224.0.0.251:mdns / Raw
Ether / IP / UDP 172.19.88.43:mdns > 224.0.0.251:mdns / Raw
Ether / IP / UDP 172.19.88.177:49685 > 255.255.255.255:10505 / Raw
Ether / IP / UDP 172.19.88.183:mdns > 224.0.0.251:mdns / Raw
Ether / IPv6 / UDP fe80::42d3:aeff:fe73:d6f8:mdns > ff02::fb:mdns / Raw
Ether / IP / UDP 172.19.85.132:mdns > 224.0.0.251:mdns / Raw
Ether / IPv6 / UDP fe80::2f4:6fff:fe96:bf79:mdns > ff02::fb:mdns / Raw
Ether / IP / UDP 172.19.98.44:35207 > 239.255.255.250:ssdp / Raw
Ether / IP / UDP 172.19.98.6:mdns > 224.0.0.251:mdns / Raw
Ether / IPv6 / UDP fe80::a684:31ff:fef5:7ca6:mdns > ff02::fb:mdns / Raw


ZombieProcess: psutil.ZombieProcess process still exists but it's a zombie (pid=60661)

In [2]:
import sys

In [1]:
string_ = 'b\'\\x17\\x03\\x03\\x009\\x00\\x00\\x00\\x00\\x00\\x00\\x05\\xba\\x04\\xaf\\xe5sB\\xdd\\xd7\\xb4-l\\xce\\xa4\\x0e\\x1dj\\n\\xc5\\xfe^4@\\x85D\\\\\\x0c\\x97\\xf5;\\x0c\\x97Z\\xbc\\x8aZHa\\xe6\\xd6'

In [4]:
str_to_bin = ''.join(format(ord(x), 'b') for x in string_)

In [5]:
print(str_to_bin)

1100010100111101110011110001100011101111011100111100011000011001110111001111000110000110011101110011110001100001100001110011011100111100011000011000010111001111000110000110000101110011110001100001100001011100111100011000011000010111001111000110000110000101110011110001100001100001011100111100011000011010110111001111000110001011000011011100111100011000011010010111001111000110000111001101011100111100011001011101011110011100001010111001111000110010011001001011100111100011001001101111011100111100011000101101001011011101100101110011110001100011110010110111001111000110000111010010111001111000110000110010110111001111000110001110010011010101011100110111010111001111000110001111010110111001111000110011011001011011110110100100000010111001111000111000110101100010010111001011100101110011110001100001100011101110011110001110011101111011100111100011001101101011110111011100111100011000011000111011100111100011100111011110110101011100111100011000101100011101110011110001110001100001101101010010001100001101

In [6]:
print("string size : ", sys.getsizeof(string_), "\nbinary size : ", sys.getsizeof(str_to_bin))

string size :  206 
binary size :  1100
